In [1]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
from lifelines import CoxPHFitter
from sklearn.model_selection import train_test_split
from sklearn.feature_selection import SelectKBest, f_classif
from lifelines.utils import concordance_index

In [2]:
npz_data = np.load('fusion_features.npz')
features = npz_data['features']  # Shape (144, 1024)
patient_ids = npz_data['patient_ids']  # Shape (144,)

event_data = pd.read_csv('processed_data.csv')

In [3]:
feature_columns = [f'feature_{i}' for i in range(features.shape[1])]
features_df = pd.DataFrame(features, columns=feature_columns)

features_df['Case ID'] = patient_ids
features_df.head()

,feature_0,feature_1,feature_2,feature_3,feature_4,feature_5,feature_6,feature_7,feature_8,feature_9,...,feature_1015,feature_1016,feature_1017,feature_1018,feature_1019,feature_1020,feature_1021,feature_1022,feature_1023,Case ID
0,0.000000,0.000000,0.000000,0.000000,0.000000,2.188303,0.000000,0.000000,0.000000,0.218016,...,0.000000,0.000000,0.000000,0.0,0.609032,0.000000,0.0,2.118962,0.0,lung_001
1,0.000000,0.000000,0.821452,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,...,0.000000,0.000000,0.000000,0.0,0.000000,1.072781,0.0,1.842743,0.0,lung_002
2,0.949904,0.019688,1.282775,0.000000,0.000000,1.265904,0.000000,0.000000,0.993333,0.000000,...,0.000000,1.956519,0.000000,0.0,0.478154,1.516294,0.0,1.078762,0.0,lung_003
3,0.189173,0.196573,1.509900,0.000000,0.350664,1.770230,0.193847,0.010835,0.000000,0.000000,...,0.391427,0.059741,1.224578,0.0,1.500176,0.213759,0.0,0.843074,0.0,lung_004
4,0.000000,0.000000,0.000000,0.842689,0.706076,0.990063,0.000000,0.000000,0.000000,0.000000,...,0.000000,0.000000,2.151799,0.0,0.000000,0.458400,0.0,0.601398,0.0,lung_005


In [4]:
event_data.head()

,Case ID,Age,Weight (lbs),Gender,Ethnicity,Smoking status,%GG,Tumor Location (choice=RUL),Tumor Location (choice=RML),Tumor Location (choice=RLL),...,Std,Min,Max,Median,SurfaceArea,Elongation,Flatness,Roundness,Time to Event,Event
0,lung_001,79,146.0,0,2,1,0,0,1,1,...,194.164635,-812,154,-56.236328,826.137989,1.436361,1.446431,0.699813,3078,0
1,lung_002,65,195.0,0,1,2,0,0,1,1,...,173.439744,-829,144,-36.021484,1037.374063,1.367921,1.112439,0.781205,70,0
2,lung_003,65,173.5,1,2,0,0,1,1,1,...,172.357348,-815,290,23.177734,755.268235,1.387373,1.089084,0.822009,666,0
3,lung_004,67,173.5,1,2,1,0,1,1,1,...,254.147443,-1024,366,21.595703,912.514223,1.593605,1.631120,0.639694,1172,0
4,lung_005,84,145.0,1,4,1,0,1,0,1,...,107.583454,-783,391,7.496094,2432.305090,1.278080,1.166290,0.735654,1456,1


In [5]:
event_data_subset = event_data[['Case ID', 'Time to Event', 'Event']]
combined_df = pd.merge(features_df, event_data_subset, on='Case ID', how='inner')
combined_df.shape[0]

144

In [6]:
features_cols = [col for col in combined_df.columns if col.startswith('feature_')]
metadata_cols = [col for col in combined_df.columns if not col.startswith('feature_')]

X = combined_df[features_cols]
y_event = combined_df['Event']
metadata = combined_df[metadata_cols]

In [7]:
k_features = 100

# Sử dụng SelectKBest với f_classif (ANOVA F-value) để chọn features tốt nhất
selector = SelectKBest(f_classif, k=k_features)
X_selected = selector.fit_transform(X, y_event)

# các features được chọn
selected_features_cols = np.array(features_cols)[selector.get_support()]
X_selected_df = pd.DataFrame(X_selected, columns=selected_features_cols)

c:\Users\ADMIN\anaconda3\envs\DATN\lib\site-packages\sklearn\feature_selection\_univariate_selection.py:111: UserWarning: Features [  76  104  150  289  307  318  319  354  394  434  440  442  463  536
  576  578  603  682  691  707  723  734  774  777  808  820  911  922
  932  939  947  960  985  998 1021] are constant.
  warnings.warn("Features %s are constant." % constant_features_idx, UserWarning)
c:\Users\ADMIN\anaconda3\envs\DATN\lib\site-packages\sklearn\feature_selection\_univariate_selection.py:112: RuntimeWarning: invalid value encountered in divide
  f = msb / msw


In [8]:
print(f"Số chiều ban đầu: {X.shape[1]}")
print(f"Số chiều sau feature selection: {X_selected_df.shape[1]}")

Số chiều ban đầu: 1024
Số chiều sau feature selection: 100


In [9]:
selected_df = pd.concat([X_selected_df, metadata], axis=1)

In [10]:
train_df, test_df = train_test_split(selected_df, test_size=0.2, random_state=42)

In [11]:
train_df = train_df.drop(columns=['Case ID'])
test_df = test_df.drop(columns=['Case ID'])

In [12]:
cph = CoxPHFitter(penalizer=0.1, l1_ratio=0.5)
cph.fit(train_df, duration_col='Time to Event', event_col='Event')

<lifelines.CoxPHFitter: fitted with 115 total observations, 73 right-censored observations>

In [13]:
cph.print_summary()  # access the individual results using cph.summary
with open("results_CoxPH_kFeat.txt", "w") as f:
    f.write(cph.summary.to_string() + "\n")
    f.write(f"Concordance Index: {cph.concordance_index_}\n")

<lifelines.CoxPHFitter: fitted with 115 total observations, 73 right-censored observations>
             duration col = 'Time to Event'
                event col = 'Event'
                penalizer = 0.1
                 l1 ratio = 0.5
      baseline estimation = breslow
   number of observations = 115
number of events observed = 42
   partial log-likelihood = -166.95
         time fit was run = 2025-04-26 16:40:29 UTC

---
              coef exp(coef)  se(coef)  coef lower 95%  coef upper 95% exp(coef) lower 95% exp(coef) upper 95%
covariate                                                                                                     
feature_0    -0.00      1.00      0.00           -0.00            0.00                1.00                1.00
feature_15   -0.00      1.00      0.00           -0.00            0.00                1.00                1.00
feature_30    0.00      1.00      0.00           -0.00            0.00                1.00                1.00
feature_38   -0.12      0.89      1.51           -3.07            2.84                0.05               17.06
feature_41    0.00      1.00      0.00           -0.00            0.00                1.00                1.00
feature_47   -0.00      1.00      0.00           -0.00            0.00                1.00                1.00
feature_50    0.00      1.00      0.00           -0.00            0.00                1.00                1.00
feature_60    0.00      1.00      0.00           -0.00            0.00                1.00                1.00
feature_72    0.00      1.00      0.00           -0.00            0.00                1.00                1.00
feature_75   -0.00      1.00      0.00           -0.00            0.00                1.00                1.00
feature_90    0.00      1.00      0.00           -0.00            0.00                1.00                1.00
feature_94   -0.00      1.00      0.00           -0.00            0.00                1.00                1.00
feature_99    0.00      1.00      0.00           -0.00            0.00                1.00                1.00
feature_100  -0.00      1.00      0.00           -0.00            0.00                1.00                1.00
feature_108  -0.00      1.00      0.00           -0.00            0.00                1.00                1.00
feature_121  -0.14      0.87      0.85           -1.80            1.52                0.16                4.55
feature_124  -0.00      1.00      0.00           -0.00            0.00                1.00                1.00
feature_136   0.27      1.31      1.97           -3.60            4.14                0.03               62.55
feature_137  -0.00      1.00      0.00           -0.00            0.00                1.00                1.00
feature_153   0.00      1.00      0.00           -0.00            0.00                1.00                1.00
feature_165   0.18      1.20      0.41           -0.62            0.98                0.54                2.68
feature_170  -0.05      0.96      0.31           -0.66            0.57                0.52                1.76
feature_175  -0.00      1.00      0.00           -0.00            0.00                1.00                1.00
feature_195  -0.00      1.00      0.00           -0.00            0.00                1.00                1.00
feature_206  -0.00      1.00      0.00           -0.00            0.00                1.00                1.00
feature_219  -0.00      1.00      0.00           -0.00            0.00                1.00                1.00
feature_232   0.00      1.00      0.00           -0.00            0.00                1.00                1.00
feature_269  -0.72      0.49      0.49           -1.69            0.25                0.18                1.28
feature_278   0.00      1.00      0.00           -0.00            0.00                1.00                1.00
feature_283   0.00      1.00      0.00           -0.00            0.00                1.00                1.00
feature_286   0.00  

In [14]:
test_ci = cph.score(test_df, scoring_method="concordance_index")
print(test_ci)
with open("results_CoxPH_kFeat.txt", "a") as f:
    f.write(f"Concordance Index on Test: {test_ci}\n")

0.7297297297297297


In [17]:
k_values = [10, 20, 50, 100, 200, 300]
ci_scores = []

for k in k_values:
    # Chọn k features
    temp_selector = SelectKBest(f_classif, k=k)
    X_temp = temp_selector.fit_transform(X, y_event)
    
    # Tạo DataFrame
    temp_cols = np.array(features_cols)[temp_selector.get_support()]
    X_temp_df = pd.DataFrame(X_temp, columns=temp_cols)
    
    # Kết hợp với metadata
    temp_df = pd.concat([X_temp_df, metadata], axis=1)
    
    # Chia tập train/test
    temp_train, temp_test = train_test_split(temp_df, test_size=0.2, random_state=42)
    
    # Loại bỏ patient_id
    temp_train = temp_train.drop(columns=['Case ID'])
    temp_test = temp_test.drop(columns=['Case ID'])
    
    # Fit mô hình
    temp_cph = CoxPHFitter(penalizer=0.1, l1_ratio=0.5)
    temp_cph.fit(temp_train, duration_col='Time to Event', event_col='Event')
    
    # Đánh giá
    ci = temp_cph.score(temp_test, scoring_method="concordance_index")
    ci_scores.append(ci)

# Vẽ biểu đồ concordance index theo số lượng features
plt.figure(figsize=(10, 6))
plt.plot(k_values, ci_scores, marker='o')
plt.xlabel('Số lượng features')
plt.ylabel('Concordance Index')
plt.title('Concordance Index theo số lượng features')
plt.grid(True)
plt.savefig('concordance_by_features.png')
plt.close()

print("\nConcordance Index theo số lượng features:")
for k, ci in zip(k_values, ci_scores):
    print(f"k={k}: CI={ci:.4f}")

c:\Users\ADMIN\anaconda3\envs\DATN\lib\site-packages\sklearn\feature_selection\_univariate_selection.py:111: UserWarning: Features [  76  104  150  289  307  318  319  354  394  434  440  442  463  536
  576  578  603  682  691  707  723  734  774  777  808  820  911  922
  932  939  947  960  985  998 1021] are constant.
  warnings.warn("Features %s are constant." % constant_features_idx, UserWarning)
c:\Users\ADMIN\anaconda3\envs\DATN\lib\site-packages\sklearn\feature_selection\_univariate_selection.py:112: RuntimeWarning: invalid value encountered in divide
  f = msb / msw
c:\Users\ADMIN\anaconda3\envs\DATN\lib\site-packages\sklearn\feature_selection\_univariate_selection.py:111: UserWarning: Features [  76  104  150  289  307  318  319  354  394  434  440  442  463  536
  576  578  603  682  691  707  723  734  774  777  808  820  911  922
  932  939  947  960  985  998 1021] are constant.
  warnings.warn("Features %s are constant." % constant_features_idx, UserWarning)
c:\Users\AD


Concordance Index theo số lượng features:
k=10: CI=0.7027
k=20: CI=0.7081
k=50: CI=0.7027
k=100: CI=0.7297
k=200: CI=0.7243
k=300: CI=0.7243
